In [21]:
import pandas as pd
import numpy as np

from get_db_url import get_db_url 
import env
host = env.host
username = env.username
password = env.password

In [22]:
url = get_db_url(username,host,password,"employees")

In [23]:
#Once you have successfully run a query:
pd.read_sql('SELECT * FROM employees', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


In [24]:
#Read the employees and titles tables into two separate DataFrames.
url = get_db_url(username,host,password,"employees")
sql = '''
SELECT
    *
FROM employees
'''
employees_df = pd.read_sql(sql, url)
sql2 = '''
SELECT
    *
FROM titles
'''
titles_df = pd.read_sql(sql2, url)


In [25]:
employees_df.to_csv('employees.csv', index = False)
#cache the data so we can access this dataframe in the future without
#having to ping the cloud database everytime
#loading it up in the future will look like
#employees = pd.read_csv('employees.csv')

In [26]:
employees_df.dtypes
type(employees_df)
employees_df.describe()
employees_df

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


In [27]:
titles_df.dtypes ,type(titles_df)
titles_df

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [28]:
#How many unique titles are in the titles DataFrame?
len(titles_df.title.value_counts())

7

In [29]:
#What is the oldest date in the to_date column?
titles_df.to_date.min()
#pd.read_sql("SELECT MIN(to_date) FROM titles", url)

datetime.date(1985, 3, 1)

In [30]:
#What is the most recent date in the to_date column?
from datetime import date
today = date.today()
titles_df[titles_df.to_date < today].max()
#titles.to_date.max
#pd.read_sql("SELECT MAX(to_date) FROM titles", url)

emp_no                 499998
title        Technique Leader
from_date          2002-06-30
to_date            2002-08-01
dtype: object

In [31]:
users_df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users_df


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [32]:

roles_df = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles_df

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [33]:
#What is the result of using a right join on the DataFrames?
pd.merge(
    users_df,
    roles_df,
    left_on="role_id",
    right_on="id",
    how="right",
    indicator=True)


,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


In [34]:
#copied for future reference
clean_right = pd.merge(users_df, 
                    roles_df, 
                    left_on='role_id', 
                    right_on='id', 
                    how='right').drop(columns='role_id').rename(columns={'id_x': 'id', 
                                                                         'name_x': 'employee',
                                                                         'id_y': 'role_id',
                                                                         'name_y': 'role'}
                                                                )
clean_right

,id,employee,role_id,role
0,1.0,bob,1,admin
1,2.0,joe,2,author
2,3.0,sally,3,reviewer
3,4.0,adam,3,reviewer
4,NaN,NaN,4,commenter


In [35]:
#What is the result of using an outer join on the DataFrames?
pd.merge(users_df,roles_df,left_on="role_id",right_on="id",how="outer",indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [36]:
#What happens if you drop the foreign keys 
# from the DataFrames and try to merge them?
pd.merge(users_df,roles_df,how="outer")
#matches (id and name) and indexes

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN
6,1,admin,NaN
7,2,author,NaN
8,3,reviewer,NaN
9,4,commenter,NaN


In [37]:
from pydataset import data
mpg_df = data("mpg")

#How many rows and columns are in the dataset?
mpg_df.shape

(234, 11)

In [38]:
#Check out your column names and perform any cleanup you may want on them.
#looks good to me currently

In [39]:
#Display the summary statistics for the dataset.
mpg_df.describe()

,displ,year,cyl,cty,hwy
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


In [40]:
#How many different manufacturers are there?
len(mpg_df.manufacturer.unique())

15

In [41]:
#How many different models are there?
len(mpg_df.model.unique())

38

In [42]:
#Create a column named mileage_difference like you did in the DataFrames 
# exercises; this column should contain the difference 
# between highway and city mileage for each car.
mpg_df["mileage_difference"] = mpg_df.hwy - mpg_df.cty

In [43]:
#Create a column named average_mileage like you did in the DataFrames 
# exercises; this is the mean of the city and highway mileage
mpg_df["average_mileage"] = ((mpg_df.hwy + mpg_df.cty)/2)
mpg_df

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0


In [44]:
#Create a new column on the mpg dataset named is_automatic 
#that holds boolean values denoting whether the car has an 
#automatic transmission.
mpg_df["is_automatic"] = (mpg_df["trans"].str.count('auto') > 0)
mpg_df

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


In [45]:
#Using the mpg dataset, find out which which manufacturer 
# has the best miles per gallon on average?
mpg_df.groupby("manufacturer")["average_mileage"].mean().nlargest(1)


manufacturer
honda    28.5
Name: average_mileage, dtype: float64

In [46]:
#Do automatic or manual cars have better miles per gallon?
mpg_df.groupby("is_automatic")["average_mileage"].mean().nlargest(1)
#manual


is_automatic
False    22.227273
Name: average_mileage, dtype: float64

In [49]:
#copied for future reference
mpg_df['transmission_category']= np.where(
    mpg_df.trans.str.contains('auto'), 'automatic', 'manual')
mpg_df.groupby('transmission_category')[['average_mileage']].mean().round(1)

,average_mileage
transmission_category,
automatic,19.1
manual,22.2


In [82]:
url2 = get_db_url(username,host,password,"chipotle")
sql = '''
SELECT
    *
FROM orders
LIMIT 10000
'''
chipotle_df = pd.read_sql(sql, url2)
chipotle_df


,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [83]:
#What is the total price for each order?
chipotle_df["item_price"] = chipotle_df.item_price.str.strip("$").astype("float")
chipotle_df.groupby("order_id")["item_price"].sum()


order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_price, Length: 1834, dtype: float64

In [84]:
#What are the most popular 3 items?
chipotle_df.groupby("item_name")["quantity"].sum().nlargest(3)

item_name
Chicken Bowl           761
Chicken Burrito        591
Chips and Guacamole    506
Name: quantity, dtype: int64

In [85]:
chipotle_df[chipotle_df["item_name"] == "Chicken Bowl"].item_price.value_counts()

8.75     313
11.25    228
8.49     104
10.98     42
17.50     11
16.98     10
21.96      7
22.50      3
8.19       3
8.50       2
32.94      1
26.25      1
10.58      1
Name: item_price, dtype: int64

In [86]:
#Which item has produced the most revenue?
chipotle_df.groupby("item_name")["item_price"].sum().nlargest(3)

item_name
Chicken Bowl       7342.73
Chicken Burrito    5575.82
Steak Burrito      3851.43
Name: item_price, dtype: float64

In [72]:
#Join the employees and titles DataFrames together
emp_titl_df = pd.merge(employees_df,titles_df,on="emp_no",how="inner")
emp_titl_df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01


In [ ]:
#For each title, find the hire date of the employee that 
# was hired most recently with that title.
emp_titl_df.groupby("title")["from_date"].max()

title
Assistant Engineer    1998-06-14
Engineer              1999-09-27
Senior Engineer       2001-04-01
Senior Staff          2002-01-31
Staff                 1999-04-30
Technique Leader      1994-05-21
Name: from_date, dtype: object

In [ ]:
url3 = get_db_url(username,host,password,"employees")
sql = '''
SELECT
    *
FROM departments
LIMIT 10000
'''
department_df = pd.read_sql(sql, url3)
department_df.head()

,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance
3,d003,Human Resources
4,d001,Marketing


In [ ]:
url4 = get_db_url(username,host,password,"employees")
sql = '''
SELECT
    *
FROM dept_emp
LIMIT 10000
'''
dept_emp_df = pd.read_sql(sql, url4)
dept_emp_df.head()

,emp_no,dept_no,from_date,to_date
0,10001,d005,1986-06-26,9999-01-01
1,10002,d007,1996-08-03,9999-01-01
2,10003,d004,1995-12-03,9999-01-01
3,10004,d004,1986-12-01,9999-01-01
4,10005,d003,1989-09-12,9999-01-01


In [ ]:
#Write the code necessary to create a cross tabulation of the 
# number of titles by department. 
# (Hint: this will involve a combination of SQL code to 
# pull the necessary data and python/pandas code to 
# perform the manipulations.)
emp_title_deptno_df = pd.merge(emp_titl_df,dept_emp_df,on="emp_no",how="inner")
emp_title_deptno_df.head()


,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date_x,to_date_x,dept_no,from_date_y,to_date_y
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,d005,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01,d007,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01,d004,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01,d004,1986-12-01,9999-01-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01,d004,1986-12-01,9999-01-01


In [ ]:
all_info_employees_df = pd.merge(emp_title_deptno_df,department_df,on="dept_no",how="inner")
all_info_employees_df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date_x,to_date_x,dept_no,from_date_y,to_date_y,dept_name
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01,d005,1986-06-26,9999-01-01,Development
1,10006,1953-04-20,Anneke,Preusig,F,1989-06-02,Senior Engineer,1990-08-05,9999-01-01,d005,1990-08-05,9999-01-01,Development
2,10008,1958-02-19,Saniya,Kalloufi,M,1994-09-15,Assistant Engineer,1998-03-11,2000-07-31,d005,1998-03-11,2000-07-31,Development
3,10012,1960-10-04,Patricio,Bridgland,M,1992-12-18,Engineer,1992-12-18,2000-12-18,d005,1992-12-18,9999-01-01,Development
4,10012,1960-10-04,Patricio,Bridgland,M,1992-12-18,Senior Engineer,2000-12-18,9999-01-01,d005,1992-12-18,9999-01-01,Development


In [ ]:
pd.crosstab(all_info_employees_df.title,all_info_employees_df.dept_name)

dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,0,2,0,0,0,1,1,0,0
Engineer,0,17,0,0,0,12,3,1,0
Senior Engineer,0,12,0,0,0,11,2,1,0
Senior Staff,3,0,3,5,2,0,0,4,5
Staff,4,0,2,3,2,0,0,5,6
Technique Leader,0,2,0,0,0,1,1,0,0
